In [6]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import gc

In [7]:
train = pd.read_pickle("../data/train_agg.pkl",compression = "gzip")
test = pd.read_pickle("../data/test_agg.pkl",compression = "gzip")

In [16]:
sub = pd.read_csv("../data/amex-default-prediction/sample_submission.csv")

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
CategoricalIndex: 458913 entries, 0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a to fffff1d38b785cef84adeace64f8f83db3a0c31e8d92eaba8b115f71cab04681
Columns: 919 entries, P_2_mean to target
dtypes: category(2), float16(713), float64(178), int64(22), int8(4)
memory usage: 1.3 GB


In [9]:
features = test.columns.to_list()
cat_features = [
    "B_30",
    "B_38",
    "D_114",
    "D_116",
    "D_117",
    "D_120",
    "D_126",
    "D_63",
    "D_64",
    "D_66",
    "D_68"
]
cat_features = [f"{cf}_last" for cf in cat_features]
le_encoder = LabelEncoder()
for categorical_feature in cat_features:
    train[categorical_feature] = le_encoder.fit_transform(train[categorical_feature])
    test[categorical_feature] = le_encoder.transform(test[categorical_feature])


In [11]:
train_y = pd.DataFrame(train["target"])
train_x = train.drop("target", axis=1)
del train
_ = gc.collect()

In [12]:
x1, x2, y1, y2 = train_test_split(train_x, train_y, test_size=0.20, random_state=22)

In [13]:
clf = CatBoostClassifier(iterations=1000, random_state=22, nan_mode='Min')
clf.fit(x1, y1, eval_set=[(x2, y2)], cat_features=cat_features,  verbose=50)
preds = clf.predict_proba(x2)[:, 1]

Learning rate set to 0.136276
0:	learn: 0.5426635	test: 0.5422141	best: 0.5422141 (0)	total: 585ms	remaining: 9m 44s
50:	learn: 0.2278892	test: 0.2288582	best: 0.2288582 (50)	total: 22.4s	remaining: 6m 56s
100:	learn: 0.2209364	test: 0.2237507	best: 0.2237507 (100)	total: 42.2s	remaining: 6m 15s
150:	learn: 0.2161837	test: 0.2215877	best: 0.2215877 (150)	total: 1m 1s	remaining: 5m 48s
200:	learn: 0.2125496	test: 0.2206460	best: 0.2206424 (199)	total: 1m 21s	remaining: 5m 22s
250:	learn: 0.2091853	test: 0.2200191	best: 0.2200191 (250)	total: 1m 41s	remaining: 5m 3s
300:	learn: 0.2061737	test: 0.2195578	best: 0.2195578 (300)	total: 2m 1s	remaining: 4m 42s
350:	learn: 0.2033721	test: 0.2193876	best: 0.2193854 (347)	total: 2m 20s	remaining: 4m 20s
400:	learn: 0.2006759	test: 0.2193602	best: 0.2193328 (399)	total: 2m 40s	remaining: 3m 59s
450:	learn: 0.1980507	test: 0.2192304	best: 0.2192122 (433)	total: 2m 59s	remaining: 3m 38s
500:	learn: 0.1954722	test: 0.2192762	best: 0.2192094 (452)	to

In [14]:
del x1,x2,y1,y2,train_x,train_y
_ = gc.collect()

In [17]:
y_preds = np.zeros(test.shape[0])
y_preds_ = np.zeros(test.shape[0])
y_preds = clf.predict_proba(test)[:, 1]
y_preds_ = sub['prediction'].to_numpy()
cust_id = sub['customer_ID'].to_numpy()
blend_preds = (y_preds_ * 0.955 + y_preds * 0.045)

In [18]:
test["prediction"] = blend_preds
test["prediction"].to_csv(f"submission1.csv", index=True)

C:\Users\JeongWoo\AppData\Local\Temp\ipykernel_24572\3358240415.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test["prediction"] = blend_preds
